In [1]:
import torch
import torchvision
import torch.nn as nn
from torch.utils.data import DataLoader

# TensorBoard: TensorFlow's Visualization Toolkit
TensorBoard provides the visualization and tooling needed for machine learning experimentation:

    · Tracking and visualizing metrics such as loss and accuracy
    · Visualizing the model graph (ops and layers)
    · Viewing histograms of weights, biases, or other tensors as they change over time
    · Projecting embeddings to a lower dimensional space
    · Displaying images, text, and audio data
    · Profiling TensorFlow programs
    · And much more

In [2]:
from torch.utils.tensorboard import SummaryWriter
from network import Network

In [3]:
train_set = torchvision.datasets.FashionMNIST(
    root='./data',
    train=True,
    transform=torchvision.transforms.Compose([
        torchvision.transforms.ToTensor()
    ])
)

In [5]:
train_loader = torch.utils.data.DataLoader(
    dataset=train_set,
    batch_size=100,
    shuffle=False
)

In [6]:
tb = SummaryWriter()

In [7]:
network = Network()
optimizer = torch.optim.Adam(
    params=network.parameters(),
    lr=0.01
)

In [8]:
images, labels = next(
    iter(train_loader)
)

In [9]:
grid = torchvision.utils.make_grid(images)

In [10]:
tb.add_image('images', grid)
tb.add_graph(network, images)

In [11]:
for epoch in range(10):

    total_loss_per_epoch = 0
    total_correct_per_epoch = 0

    for batch in train_loader:

        images, labels = batch # 1. Get Batch

        predictions = network(images) # 2. Pass Batch into Network

        loss = nn.functional.cross_entropy(
            input=predictions,
            target=labels
        ) # 3. Calculate Loss Function

        optimizer.zero_grad() # 4A. Zero out the gradients
        loss.backward() # 4B. Calculate the gradients
        optimizer.step() # 5. Update the weights

        total_loss_per_epoch += loss.item()
        total_correct_per_epoch += predictions.argmax(dim=1).eq(labels).sum().item()
        # 6. Iterate over all batches from the dataset

    print(
        'Epoch:', epoch,
        '\nTotal_correct:', total_correct_per_epoch,
        '\nTotal_loss:', total_loss_per_epoch,
        '\nAccuracy:', total_correct_per_epoch/len(train_set)
    )
    tb.add_scalar('Loss', total_loss_per_epoch, epoch)
    tb.add_scalar('Correct', total_correct_per_epoch, epoch)
    tb.add_scalar('Accuracy', total_correct_per_epoch/len(train_set))

    tb.add_histogram('conv1.bias', network.conv1.bias, epoch)
    tb.add_histogram('conv1.weight', network.conv1.weight, epoch)
    tb.add_histogram('conv1.weight.grad', network.conv1.weight.grad, epoch)
    # 7. Iterate through next epochs until the loss function is minimized

Epoch: 0 
Total_correct: 46901 
Total_loss: 348.6813206523657 
Accuracy: 0.7816833333333333
Epoch: 1 
Total_correct: 51425 
Total_loss: 231.83927330374718 
Accuracy: 0.8570833333333333
Epoch: 2 
Total_correct: 52113 
Total_loss: 214.81694144010544 
Accuracy: 0.86855
Epoch: 3 
Total_correct: 52398 
Total_loss: 207.53233568370342 
Accuracy: 0.8733
Epoch: 4 
Total_correct: 52786 
Total_loss: 196.99488213658333 
Accuracy: 0.8797666666666667
Epoch: 5 
Total_correct: 52994 
Total_loss: 190.42453299462795 
Accuracy: 0.8832333333333333
Epoch: 6 
Total_correct: 52980 
Total_loss: 190.9007575660944 
Accuracy: 0.883
Epoch: 7 
Total_correct: 53184 
Total_loss: 184.1041710227728 
Accuracy: 0.8864
Epoch: 8 
Total_correct: 53263 
Total_loss: 184.73317089676857 
Accuracy: 0.8877166666666667
Epoch: 9 
Total_correct: 53243 
Total_loss: 182.4922836869955 
Accuracy: 0.8873833333333333


In [12]:
tb.close()